In [ ]:
import pandas as pd
import numpy as np

src = "data/Fire_For16-21_Attributes.csv"
df = pd.read_csv(src)

fire_cols = ["FIRE_1617","FIRE_1718","FIRE_1819","FIRE_1920","FIRE_2021"]
df = df[["STATE","FOREST","COUNT"] + fire_cols].copy()
df = df[df["FOREST"] == 1]  # forest only

long = df.melt(id_vars=["STATE","COUNT"], value_vars=fire_cols,
               var_name="season", value_name="code")

season_map = {
    "FIRE_1617": "2017",
    "FIRE_1718": "2018",
    "FIRE_1819": "2019",
    "FIRE_1920": "2020",
    "FIRE_2021": "2021",
}
long["year"] = long["season"].map(season_map)

long["code"] = long["code"].fillna(" ")
long["is_unplanned"] = (long["code"] == "U").astype(int)
long["is_planned"] = (long["code"] == "P").astype(int)

grp = long.groupby(["STATE","year"]).agg(
    pixels_forest=("COUNT","sum"),
    pixels_unplanned=("is_unplanned", lambda s: np.sum(s * long.loc[s.index, "COUNT"])),
    pixels_planned=("is_planned",   lambda s: np.sum(s * long.loc[s.index, "COUNT"])),
).reset_index()

grp["pct_unplanned_of_forest"] = 100 * grp["pixels_unplanned"] / grp["pixels_forest"]
grp["pct_planned_of_forest"]   = 100 * grp["pixels_planned"]   / grp["pixels_forest"]
grp.to_csv("fire_state_year.csv", index=False)

nat = grp.groupby("year").agg(
    pixels_forest=("pixels_forest","sum"),
    pixels_unplanned=("pixels_unplanned","sum"),
    pixels_planned=("pixels_planned","sum"),
).reset_index()
nat["pct_unplanned_of_forest"] = 100 * nat["pixels_unplanned"] / nat["pixels_forest"]
nat["pct_planned_of_forest"]   = 100 * nat["pixels_planned"]   / nat["pixels_forest"]
nat.to_csv("fire_national_year.csv", index=False)

base = grp.groupby("STATE").agg(total_forest_pixels=("pixels_forest","sum")).reset_index()
base.to_csv("fire_state_forest_totals.csv", index=False)


In [1]:
import pandas as pd
df = pd.read_csv("data/fire_state_year.csv")
df["STATE_YEAR"] = df["STATE"].astype(str) + "-" + df["year"].astype(str)
df.to_csv("data/fire_state_year.csv", index=False)
